In [2]:
# Step 1: Load the necessary libraries
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import mixed_precision

In [ ]:
# Step 2: Check for GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Enable mixed precision training (if supported by your GPU)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

Num GPUs Available:  0


In [12]:
# Step 3: Load the dataset (assuming it's saved as a JSON file)
df = pd.read_json('../../dataset/News_Category_Dataset_v3.json', lines = True) 
df

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [13]:
# Step 4: Keep both 'title' and 'short_description' columns
df = df[['headline', 'short_description', 'category']]


In [14]:

# Step 5: Find the top 10 most frequent categories
top_categories = df['category'].value_counts().head(10).index


In [15]:
# Step 6: Filter the dataset to include only the top 10 frequent categories
df_filtered = df[df['category'].isin(top_categories)]

In [17]:
# Step 7: Combine 'title' and 'short_description' into one text column
df_filtered['text'] = df_filtered['headline'] + " " + df_filtered['short_description']

C:\Users\71526\AppData\Local\Temp\ipykernel_5516\466674105.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['text'] = df_filtered['headline'] + " " + df_filtered['short_description']


In [19]:
# Step 8: Initialize the Sentence-Transformer model for sentence embeddings and move to GPU
model = SentenceTransformer('all-MiniLM-L6-v2')  # Use GPU if available

In [20]:
# Step 9: Generate sentence embeddings for each row in the 'text' column
X = model.encode(df_filtered['text'].tolist(), convert_to_tensor=False)  # List of sentence embeddings


In [21]:
# Step 10: Encode the target labels ('category') using LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df_filtered['category'])
y = to_categorical(y)  # Convert labels to one-hot encoded format

In [22]:
# Step 11: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Step 12: Build the Keras neural network model
model_nn = Sequential()

In [24]:
# Add input layer (Dense layer with 512 units)
model_nn.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))

C:\Users\71526\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# Add dropout layer for regularization
model_nn.add(Dropout(0.5))

In [26]:
# Add hidden layer
model_nn.add(Dense(256, activation='relu'))

# Add dropout layer for regularization
model_nn.add(Dropout(0.5))

# Add output layer with softmax activation (for multi-class classification)
model_nn.add(Dense(y_train.shape[1], activation='softmax'))

In [27]:
# Step 13: Compile the model
model_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
# Step 14: Train the model
history = model_nn.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.7090 - loss: 0.8983 - val_accuracy: 0.7989 - val_loss: 0.5910
Epoch 2/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.8006 - loss: 0.6115 - val_accuracy: 0.8110 - val_loss: 0.5617
Epoch 3/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.8132 - loss: 0.5676 - val_accuracy: 0.8154 - val_loss: 0.5442
Epoch 4/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.8203 - loss: 0.5403 - val_accuracy: 0.8165 - val_loss: 0.5378
Epoch 5/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8296 - loss: 0.5145 - val_accuracy: 0.8227 - val_loss: 0.5267
Epoch 6/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.8349 - loss: 0.4971 - val_accuracy: 0.8232 - val_loss: 0.5267
Epoch 7/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8400 - loss: 0.4743 - val_accuracy: 0.8225 - val_loss: 0.5252
Epoch 8/10
1560/1560 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.8459 - 

In [29]:
# Step 15: Evaluate the model
loss, accuracy = model_nn.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

780/780 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8196 - loss: 0.5389
Test Loss: 0.5284491181373596
Test Accuracy: 0.8248257040977478
